# graphblas.matrix_multiply

This example will go over how to use the `--graphblas-lower` pass from `graphblas-opt` to lower the `graphblas.matrix_multiply` op.

Let’s first import some necessary modules and generate an instance of our JIT engine.

In [1]:
import mlir_graphblas
import mlir_graphblas.sparse_utils
import numpy as np

engine = mlir_graphblas.MlirJitEngine()

Here are the passes we'll use.

In [2]:
passes = [
    "--graphblas-lower",
    "--sparsification",
    "--sparse-tensor-conversion",
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-std-to-llvm",
]

Similar to our examples using the GraphBLAS dialect, we'll need some helper functions to convert sparse tensors to dense tensors. 

We'll also need some helpers to convert our sparse matrices to CSC format. 

In [3]:
mlir_text = """
#trait_densify_csr = {
  indexing_maps = [
    affine_map<(i,j) -> (i,j)>,
    affine_map<(i,j) -> (i,j)>
  ],
  iterator_types = ["parallel", "parallel"]
}

#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csr_densify4x4(%argA: tensor<4x4xf64, #CSR64>) -> tensor<4x4xf64> {
  %output_storage = constant dense<0.0> : tensor<4x4xf64>
  %0 = linalg.generic #trait_densify_csr
    ins(%argA: tensor<4x4xf64, #CSR64>)
    outs(%output_storage: tensor<4x4xf64>) {
      ^bb(%A: f64, %x: f64):
        linalg.yield %A : f64
    } -> tensor<4x4xf64>
  return %0 : tensor<4x4xf64>
}

#trait_densify_csc = {
  indexing_maps = [
    affine_map<(i,j) -> (j,i)>,
    affine_map<(i,j) -> (i,j)>
  ],
  iterator_types = ["parallel", "parallel"]
}

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csc_densify4x4(%argA: tensor<4x4xf64, #CSC64>) -> tensor<4x4xf64> {
  %output_storage = constant dense<0.0> : tensor<4x4xf64>
  %0 = linalg.generic #trait_densify_csc
    ins(%argA: tensor<4x4xf64, #CSC64>)
    outs(%output_storage: tensor<4x4xf64>) {
      ^bb(%A: f64, %x: f64):
        linalg.yield %A : f64
    } -> tensor<4x4xf64>
  return %0 : tensor<4x4xf64>
}

func @convert_csr_to_csc(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSC64> {
    %answer = graphblas.convert_layout %sparse_tensor : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSC64>
    return %answer : tensor<?x?xf64, #CSC64>
}
"""

Let's compile our MLIR code. 

In [4]:
engine.add(mlir_text, passes)

['csr_densify4x4', 'csc_densify4x4', 'convert_csr_to_csc']

## Overview of graphblas.matrix_multiply

Here, we'll show how to use the `graphblas.matrix_multiply` op. 

`graphblas.matrix_multiply` takes a sparse matrix operand in CSR format, a sparse matrix operand in CSC format, and a `semiring` attribute. 

The single `semiring` attribute indicates an element-wise operator and an aggregation operator. For example, the plus-times semiring indicates an element-wise operator of multiplication and an aggregation operator of addition/summation. For more details about semirings, see [here](https://en.wikipedia.org/wiki/GraphBLAS).

`graphblas.matrix_multiply` applies the semiring's element-wise operator and aggregation operator in matrix-multiply order over the two given sparse matrices. For example, using `graphblas.matrix_multiply` with the plus-times semiring will get a matrix that is the result of a conventional matrix multiply.


Here's an example use of the `graphblas.matrix_multiply` op:
```
%answer = graphblas.matrix_multiply %argA, %argB, %mask { semiring = "plus_times" } : (tensor<2x2xf64, #CSR64>, tensor<2x2xf64, #CSC64>, tensor<2x2xf64, #CSR64>) to tensor<2x2xf64, #CSR64>
```

The supported options for the `semiring` attribute are "plus_pair", "plus_plus", and "plus_times".

`graphblas.matrix_multiply` can also take an optional mask operand (a CSR matrix) as shown in this example:
```
%answer = graphblas.matrix_multiply %argA, %argB, %mask { semiring = "plus_times" } : (tensor<2x3xf64, #CSR64>, tensor<3x2xf64, #CSC64>, tensor<2x2xf64, #CSR64>) to tensor<2x2xf64, #CSR64>
```

The mask operand must have the same shape as the output matrix. The mask operand acts as a boolean mask (though doesn't necessarily have to have a boolean element type) for the result, which increases performance since the mask will indicate which values in the output do not have to be calculated.

`graphblas.matrix_multiply` can also take an optional [region](https://mlir.llvm.org/docs/LangRef/#regions) as shown in this example:
```
%cf4 = constant 4.0 : f64
%answer = graphblas.matrix_multiply %argA, %argB { semiring = "plus_times" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64> {
        ^bb0(%value: f64):
            %result = std.addf %value, %cf4: f64
            graphblas.yield %result : f64
    }
```
The NumPy equivalent of this code would be `answer = (argA @ argB) + 4.0`.

The region specifies element-wise post-processing done on values that survived the masking (applies to all elements if no mask). We'll go into deeper details later on on how to write a region using `graphblas.yield`. 

Let's create some example input matrices.

In [5]:
indices = np.array(
    [
        [0, 3],
        [1, 3],
        [2, 0],
        [3, 0],
        [3, 1],
    ],
    dtype=np.uint64,
)
values = np.array([1, 2, 3, 4, 5], dtype=np.float64)
sizes = np.array([4, 4], dtype=np.uint64)
sparsity = np.array([False, True], dtype=np.bool8)

A = mlir_graphblas.sparse_utils.MLIRSparseTensor(indices, values, sizes, sparsity)

In [6]:
indices = np.array(
    [
        [0, 1],
        [0, 3],
        [1, 1],
        [1, 3],
        [2, 0],
        [2, 2],
        [3, 0],
        [3, 2],
    ],
    dtype=np.uint64,
)
values = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=np.float64)
sizes = np.array([4, 4], dtype=np.uint64)
sparsity = np.array([False, True], dtype=np.bool8)

B_csr = mlir_graphblas.sparse_utils.MLIRSparseTensor(indices, values, sizes, sparsity)
B = engine.convert_csr_to_csc(B_csr)

In [7]:
indices = np.array(
    [
        [0, 1],
        [0, 2],
        [1, 1],
        [1, 2],
        [2, 1],
        [2, 2],
        [3, 1],
        [3, 2],
    ],
    dtype=np.uint64,
)
values = np.array([-0.1, 0.2, -0.3, 0.4, -0.5, 0.6, -0.7, 0.8], dtype=np.float64)
sizes = np.array([4, 4], dtype=np.uint64)
sparsity = np.array([False, True], dtype=np.bool8)

mask = mlir_graphblas.sparse_utils.MLIRSparseTensor(indices, values, sizes, sparsity)

In [8]:
A_dense = engine.csr_densify4x4(A)

In [9]:
A_dense

array([[0., 0., 0., 1.],
       [0., 0., 0., 2.],
       [3., 0., 0., 0.],
       [4., 5., 0., 0.]])

In [10]:
B_dense = engine.csc_densify4x4(B)

In [11]:
B_dense

array([[0., 1., 0., 2.],
       [0., 3., 0., 4.],
       [5., 0., 6., 0.],
       [7., 0., 8., 0.]])

In [12]:
mask_dense = engine.csr_densify4x4(mask)

In [13]:
mask_dense

array([[ 0. , -0.1,  0.2,  0. ],
       [ 0. , -0.3,  0.4,  0. ],
       [ 0. , -0.5,  0.6,  0. ],
       [ 0. , -0.7,  0.8,  0. ]])

## graphblas.matrix_multiply (Plus-Times Semiring)

Here, we'll simply perform a conventional matrix-multiply by using `graphblas.matrix_multiply` with the plus-times semiring.

In [14]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

module {
    func @matrix_multiply_plus_times(%a: tensor<?x?xf64, #CSR64>, %b: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSR64> {
        %answer = graphblas.matrix_multiply %a, %b { semiring = "plus_times" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64>
        return %answer : tensor<?x?xf64, #CSR64>
    }
}
"""

In [15]:
engine.add(mlir_text, passes)

['matrix_multiply_plus_times']

In [16]:
sparse_matmul_result = engine.matrix_multiply_plus_times(A, B)

In [17]:
engine.csr_densify4x4(sparse_matmul_result)

array([[ 7.,  0.,  8.,  0.],
       [14.,  0., 16.,  0.],
       [ 0.,  3.,  0.,  6.],
       [ 0., 19.,  0., 28.]])

The result looks sane. Let's verify that it has the same behavior as NumPy.

In [18]:
np.all(A_dense @ B_dense == engine.csr_densify4x4(sparse_matmul_result))

True

## graphblas.matrix_multiply (Plus-Plus Semiring with Mask)

Here, we'll perform a matrix-multiply with the plus-plus semiring. We'll show the result with and without a mask to demonstrate how the masking works.

In [19]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

module {
    func @matrix_multiply_plus_plus_no_mask(%a: tensor<?x?xf64, #CSR64>, %b: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSR64> {
        %answer = graphblas.matrix_multiply %a, %b { semiring = "plus_plus" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64>
        return %answer : tensor<?x?xf64, #CSR64>
    }
    
    func @matrix_multiply_plus_plus(%a: tensor<?x?xf64, #CSR64>, %b: tensor<?x?xf64, #CSC64>, %m: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSR64> {
        %answer = graphblas.matrix_multiply %a, %b, %m { semiring = "plus_plus" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>, tensor<?x?xf64, #CSR64>) to tensor<?x?xf64, #CSR64>
        return %answer : tensor<?x?xf64, #CSR64>
    }
}
"""

In [20]:
engine.add(mlir_text, passes)

['matrix_multiply_plus_plus_no_mask', 'matrix_multiply_plus_plus']

In [21]:
no_mask_result = engine.matrix_multiply_plus_plus_no_mask(A, B)
with_mask_result = engine.matrix_multiply_plus_plus(A, B, mask)

In [22]:
engine.csr_densify4x4(no_mask_result)

array([[ 8.,  0.,  9.,  0.],
       [ 9.,  0., 10.,  0.],
       [ 0.,  4.,  0.,  5.],
       [ 0., 13.,  0., 15.]])

In [23]:
engine.csr_densify4x4(with_mask_result)

array([[ 0.,  0.,  9.,  0.],
       [ 0.,  0., 10.,  0.],
       [ 0.,  4.,  0.,  0.],
       [ 0., 13.,  0.,  0.]])

Note how the results in the masked output only have elements present in the positions where the mask had elements present. 

Since we can't verify the results via NumPy given that it doesn't support semirings in its matrix multiply implementation, we'll leave the task of verifying the results as an exercise for the reader. Note that if we're applying the element-wise operation to the values at two positions (one each sparse tensor) and one position has a value but not the other does not, then the element-wise operation for these two positions will contribute no value to be aggregated.

## graphblas.matrix_multiply (Plus-Pair Semiring with Region)

Here, we'll perform a matrix-multiply with the plus-pair semiring. We'll show the result without using a region and with a region. 

The element-wise operation of the plus-pair semiring is defined as `pair(x, y) = 1`.

In [24]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

module {
    func @matrix_multiply_plus_pair_no_region(%a: tensor<?x?xf64, #CSR64>, %b: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSR64> {
        %answer = graphblas.matrix_multiply %a, %b { semiring = "plus_pair" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64>
        return %answer : tensor<?x?xf64, #CSR64>
    }
    
    func @matrix_multiply_plus_pair_and_square(%a: tensor<?x?xf64, #CSR64>, %b: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSR64> {
        %answer = graphblas.matrix_multiply %a, %b { semiring = "plus_pair" } : (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSC64>) to tensor<?x?xf64, #CSR64> {
            ^bb0(%value: f64):
                %result = std.mulf %value, %value: f64
                graphblas.yield %result : f64
        }
        return %answer : tensor<?x?xf64, #CSR64>
    }
}
"""

In [25]:
engine.add(mlir_text, passes)

['matrix_multiply_plus_pair_no_region', 'matrix_multiply_plus_pair_and_square']

The code in the region of `matrix_multiply_plus_pair_and_square` simply squares each individual element's value. The use of `graphblas.yield` is used here to indicate the result of each element-wise squaring.

Let's first get our results without the region. 

`matrix_multiply_plus_pair_no_region` simply does a matrix multiply with the plus-pair semiring.

In [26]:
no_region_result = engine.matrix_multiply_plus_pair_no_region(A, B)

In [27]:
engine.csr_densify4x4(no_region_result)

array([[1., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 1.],
       [0., 2., 0., 2.]])

Let's now get the results from `matrix_multiply_plus_pair_and_square`.

In [28]:
with_region_result = engine.matrix_multiply_plus_pair_and_square(A, B)

In [29]:
engine.csr_densify4x4(with_region_result)

array([[1., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 1.],
       [0., 4., 0., 4.]])

Let's verify that our results are sane.

In [30]:
np.all(engine.csr_densify4x4(with_region_result) == engine.csr_densify4x4(no_region_result)**2)

True